In [1]:
import pyperclip

In [2]:
import pymsgbox

In [3]:

from transformers import pipeline
import time

p:\pyperclick\pyperclick\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [12]:

def summarize_clipboard_text(text):
    if len(text) > 100:  # Assuming text > 100 characters needs summarization
        summary = summarizer(text, max_length=50, min_length=10, do_sample=False)
        pymsgbox.alert(summary[0]['summary_text'], "Summary")
    else:
        pymsgbox.alert("The selected text is too short to summarize.", "Error")

def monitor_clipboard():
    last_text = ""
    print("Monitoring clipboard for text...")
    while True:
        text = pyperclip.paste()
        if text != last_text:  # Detect new text
            last_text = text
            summarize_clipboard_text(text)
        time.sleep(1)  # Check the clipboard every second

monitor_clipboard()


Monitoring clipboard for text...


KeyboardInterrupt: 

# Attempt to make a Stand Alone sharebale EXE

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
from pystray import Icon, Menu, MenuItem
from PIL import Image, ImageDraw

In [6]:
import threading

In [7]:
# Function to create a simple tray icon
def create_image():
    width, height = 64, 64
    color1, color2 = "white", "black"
    image = Image.new("RGB", (width, height), color1)
    dc = ImageDraw.Draw(image)
    dc.rectangle((width // 4, height // 4, width * 3 // 4, height * 3 // 4), fill=color2)
    return image

In [8]:
# Clipboard monitoring flag
monitoring = False

In [12]:
# Load the model and tokenizer locally
def load_summarizer():
    print("Loading summarization model...")
    model_name = "t5-small"  # Lightweight model for summarization
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_summarizer()

Loading summarization model...


In [10]:
def summarize_text(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
def summarize_clipboard_text():
    text = pyperclip.paste()
    if len(text) > 100:  # Assuming text > 100 characters needs summarization
        summary = summarize_text(text)
        pymsgbox.alert(summary, "Summary")
    else:
        pymsgbox.alert("The selected text is too short to summarize.", "Error")

In [12]:
def monitor_clipboard():
    global monitoring
    monitoring = True
    last_text = ""
    print("Monitoring clipboard for text...")
    while True:
        text = pyperclip.paste()
        if text != last_text:  # Detect new text
            last_text = text
            summarize_clipboard_text()
        time.sleep(1)  # Check the clipboard every second

In [13]:
# Menu actions for the tray icon
def on_start(icon, item):
    threading.Thread(target=monitor_clipboard, daemon=True).start()

def on_pause(icon, item):
    global monitoring
    monitoring = False

def on_exit(icon, item):
    icon.stop()

In [ ]:
# Create the system tray icon and menu
menu = Menu(
    MenuItem("Start Monitoring", on_start),
    MenuItem("Pause Monitoring", on_pause),
    MenuItem("Exit", on_exit)
)
tray_icon = Icon("Clipboard Summarizer", create_image(), "Clipboard Summarizer", menu)

In [ ]:
tray_icon.run()

Monitoring clipboard for text...


Exception ignored on calling ctypes callback function: <function _dispatcher at 0x000002672E033C40>
Traceback (most recent call last):
  File "p:\pyperclick\pyperclick\Lib\site-packages\pystray\_win32.py", line 395, in _dispatcher
    @win32.WNDPROC
    
KeyboardInterrupt: 


# Trying something better

In [7]:
from plyer import notification

In [8]:
import os

In [9]:
def notify_user(title, message):
    notification.notify(
        title=title,
        message=message,
        app_name="Clipboard Summarizer",
        timeout=5
    )

In [10]:
# Clipboard monitoring flag
monitoring = threading.Event()

In [11]:
# Load the model and tokenizer locally
def load_summarizer():
    print("Loading summarization model...")
    model_name = "t5-small"  # Lightweight model for summarization
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

In [12]:
try:
    tokenizer, model = load_summarizer()
except Exception as e:
    print("Error loading summarization model:", e)
    pymsgbox.alert("Failed to load summarization model. Please check your installation.", "Error")
    exit()

Loading summarization model...


In [13]:
def summarize_text(text):
    max_input_length = tokenizer.model_max_length
    if len(text) > max_input_length:
        text = text[:max_input_length]
        print("Input text truncated to fit model limits.")
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [14]:
def summarize_clipboard_text():
    try:
        text = pyperclip.paste()
        if not isinstance(text, str):
            raise ValueError("Clipboard content is not text.")
        if len(text) > 100:
            summary = summarize_text(text)
            notify_user("Summary", summary)
        else:
            notify_user("Error", "The selected text is too short to summarize.")
    except Exception as e:
        print("Error summarizing clipboard text:", e)

In [15]:
def monitor_clipboard():
    monitoring.set()
    last_text = ""
    print("Monitoring clipboard for text...")
    while monitoring.is_set():
        try:
            text = pyperclip.paste()
        except Exception as e:
            print("Error reading clipboard:", e)
            time.sleep(1)
            continue
        if text != last_text:  # Detect new text
            last_text = text
            summarize_clipboard_text()
        time.sleep(1)  # Check the clipboard every second

In [16]:

# Menu actions for the tray icon
def on_start(icon, item):
    if not monitoring.is_set():
        notify_user("Clipboard monitoring started.")
        threading.Thread(target=monitor_clipboard, daemon=True).start()
    else:
        notify_user("Monitoring is already running.")

def on_pause(icon, item):
    monitoring.clear()
    notify_user("Clipboard monitoring paused.")

def on_exit(icon, item):
    monitoring.clear()
    icon.stop()

# Validate icon path
icon_path = "clipboard.ico"
if not os.path.exists(icon_path):
    print(f"Icon file '{icon_path}' not found. Using default icon.")
    icon_path = None

# Load the icon image
def load_icon(icon_path):
    try:
        return Image.open(icon_path)
    except Exception as e:
        print(f"Failed to load icon: {e}")
        return None
    
# Load the icon image
icon_image = load_icon("clipboard.ico")
if not icon_image:
    print("Could not load the tray icon. Exiting.")
    exit()
# Create the system tray icon and menu
menu = Menu(
    MenuItem("Start Monitoring", on_start),
    MenuItem("Pause Monitoring", on_pause),
    MenuItem("Exit", on_exit)
)
tray_icon = Icon("Clipboard Summarizer", icon_image, "Clipboard Summarizer", menu)


In [ ]:
tray_icon.run()

An error occurred when calling message handler
Traceback (most recent call last):
  File "p:\pyperclick\pyperclick\Lib\site-packages\pystray\_win32.py", line 412, in _dispatcher
    return int(icon._message_handlers.get(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "p:\pyperclick\pyperclick\Lib\site-packages\pystray\_win32.py", line 224, in _on_notify
    descriptors[index - 1](self)
  File "p:\pyperclick\pyperclick\Lib\site-packages\pystray\_base.py", line 328, in inner
    callback(self)
  File "p:\pyperclick\pyperclick\Lib\site-packages\pystray\_base.py", line 453, in __call__
    return self._action(icon, self)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alok\AppData\Local\Temp\ipykernel_3304\2390507762.py", line 4, in on_start
    notify_user("Clipboard monitoring started.")
TypeError: notify_user() missing 1 required positional argument: 'message'
